In [2]:
%load_ext autoreload
%autoreload 2   
import carla
import math
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
client = carla.Client('localhost', 2000)
world = client.get_world()
world_map = world.get_map()
settings = world.get_settings()
settings.fixed_delta_seconds = None
settings.synchronous_mode = False
world.apply_settings(settings)
world.wait_for_tick()
spawn_points = world_map.get_spawn_points()
start_point = spawn_points[0]

In [4]:
# Get the blueprint of a black car 
blueprint_library = world.get_blueprint_library()
vehicle_bp = random.choice(blueprint_library.filter('vehicle.toyota.prius'))
vehicle_bp.set_attribute('color', '0,0,0')


In [5]:
# Spawn a vehicle on where I am looking at

# first get where I am looking at
# spectator = world.get_spectator()
# spectator_transform = spectator.get_transform()
# location = spectator_transform.location
# rotation = spectator_transform.rotation





# distance_ahead = 10  # Spawn the car 10 meters in front
# yaw = math.radians(rotation.yaw)  # Convert yaw to radians

# spawn_location = carla.Location(
#     x=location.x + distance_ahead * math.cos(yaw),
#     y=location.y + distance_ahead * math.sin(yaw),
#     z=location.z  # Keep the Z-height same (adjust if needed)
# )

# vehicle = world.try_spawn_actor(vehicle_bp, start_point)h
# spawn_transform = carla.Transform(spawn_location, rotation)
# jake = world.try_spawn_actor(vehicle_bp, spawn_transform)

# Spawn the vehicle
# jake = world.try_spawn_actor(vehicle_bp, spawn_transform)
# Spawn a vehicle on where I am looking at

In [6]:

def destroy_all_prius(world):
    actors = world.get_actors()
    priuses = [actor for actor in actors if 'vehicle.toyota.prius' in actor.type_id]

    for prius in priuses:
        prius.destroy()

# destroy_all_prius(world)




In [7]:
# spawn jake at the left side of the parking lot 
destroy_all_prius(world)

def spawn_jake():
    jake_location = carla.Location(x=-58.468666, y=188.412048, z=0.1)
    jake_rotation = carla.Rotation(pitch=0, yaw=-90, roll=0) 

    jake_transform = carla.Transform(jake_location, jake_rotation)
    # Spawn the camera at the spectator's position and orientation
    jake = world.spawn_actor(vehicle_bp, jake_transform)
    return jake

jake = spawn_jake()


In [8]:
# # Spawn a BEV camera and segment stuff using deeplabv3

# import cv2
# import numpy as np
# import torch
# model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)
# model.eval()
# COLORS = np.array([
#     [255, 0, 0], [0, 255, 0], [0, 0, 255], [128, 128, 0], [128, 0, 128], [0, 128, 128], 
#     [128, 64, 64], [64, 128, 128], [64, 128, 64], [64, 64, 196], [196, 128, 0], [196, 128, 196],
#     # Add more colors as needed for other classes
# ])

# # create the BEV camera above the parking lot 
# camera_bp = blueprint_library.find('sensor.camera.rgb')
# camera_bp.set_attribute('image_size_x', '800')
# camera_bp.set_attribute('image_size_y', '600')
# camera_bp.set_attribute('fov', '90')  # Adjust FOV if needed
# camera_bp.set_attribute('sensor_tick', '3.0')
# bev_location = carla.Location(x=-58.686981, y=166.857986, z=24.152607) 
# spectator_rotation = carla.Rotation(pitch=-90, yaw=-180, roll=0) 

# # Create a transform using the given location and rotation
# bev_transform = carla.Transform(bev_location, spectator_rotation)
# # Spawn the camera at the spectator's position and orientation
# camera = world.spawn_actor(camera_bp, bev_transform)

# def process_image(image):
#     array = np.frombuffer(image.raw_data, dtype=np.uint8)
#     array = array.reshape((image.height, image.width, 4))  # Convert to 2D array
#     image_rgb = array[:, :, :3]  # Discard the alpha channel

#     # Preprocess image for DeepLabv3 (resize and normalize)
#     input_image = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2RGB)
#     input_image = torch.tensor(input_image).float().permute(2, 0, 1).unsqueeze(0) / 255.0

#     # Run the model on the image
#     with torch.no_grad():
#         output = model(input_image)['out'][0]  # Get model's output
#     output_predictions = output.argmax(0)  # Get the class predictions (highest logit)
    
#     # Convert the predictions to colors
#     seg_image = COLORS[output_predictions.numpy()]
    
#     # Create the masked image by blending the segmentation output with the original image
#     masked_image = np.copy(image_rgb)
#     masked_image[output_predictions.numpy() == 0] = [0, 0, 0]  # Optionally mask out background (class 0)

#     # Overlay the segmentation result on the original image
#     blended_image = cv2.addWeighted(image_rgb, 0.7, seg_image.astype(np.uint8), 0.3, 0)
#     unique_values = np.unique(output_predictions)
#     # Display the result
#     cv2.imshow("BEV + Segmentation", blended_image)
#     cv2.waitKey(1)
    


In [9]:
# # Spawn a camera in front of jake and do segmenation using YOLO
# import cv2
# import numpy as np
# import threading
# # from yolo import *
# from ultralytics import YOLO
# seg_model = YOLO("yolov8n.pt") 


# # create the BEV camera above the parking lot 
# camera_bp = blueprint_library.find('sensor.camera.rgb')
# camera_bp.set_attribute('image_size_x', '800')
# camera_bp.set_attribute('image_size_y', '600')
# camera_bp.set_attribute('fov', '90')  # Adjust FOV if needed
# # camera_bp.set_attribute('sensor_tick', '5.0')
# camera_transform = carla.Transform(carla.Location(x=0, y=5, z=2))  # Adjust if needed

# camera = world.spawn_actor(camera_bp, camera_transform, attach_to=jake)

# def process_image(image):
#     array = np.frombuffer(image.raw_data, dtype=np.uint8)
#     array = array.reshape((image.height, image.width, 4))  # Convert to 2D array
#     array = array[:, :, :3]  # Remove alpha channel
#     results = seg_model(array)  # Perform inference
#     annotated_frame = results[0].plot()  # Get the annotated image
#     cv2.imshow("BEV + Segmentation", annotated_frame)
    
#     # Check if window is closed
#     # key = cv2.waitKey(1) & 0xFF
#     # if cv2.getWindowProperty("BEV + Segmentation", cv2.WND_PROP_VISIBLE) < 1:
#     #     running = False
#     #     camera.stop()
#     #     cv2.destroyAllWindows()

#     cv2.waitKey(1)
# camera.listen(lambda image: process_image(image))

In [10]:
# # Spawn a camera in front of jake and do segmenation using DeepLabv3
# import cv2
# import numpy as np
# import torch
# import threading 
# model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)
# model.eval()
# COLORS = np.array([
#     [255, 0, 0], [0, 255, 0], [0, 0, 255], [128, 128, 0], [128, 0, 128], [0, 128, 128], 
#     [128, 64, 64], [64, 128, 128], [64, 128, 64], [64, 64, 196], [196, 128, 0], [196, 128, 196],
#     # Add more colors as needed for other classes
# ])



# # create the BEV camera above the parking lot 
# camera_bp = blueprint_library.find('sensor.camera.rgb')
# camera_bp.set_attribute('image_size_x', '800')
# camera_bp.set_attribute('image_size_y', '600')
# camera_bp.set_attribute('fov', '90')  # Adjust FOV if needed
# # camera_bp.set_attribute('sensor_tick', '1.0')
# camera_bp.set_attribute('sensor_tick', '0.033333')  # 30 FPS 
# # camera_bp.set_attribute('sensor_tick', '5.0')
# camera_transform = carla.Transform(carla.Location(x=0, y=0, z=2))  # Adjust if needed

# camera = world.spawn_actor(camera_bp, camera_transform, attach_to=jake)

# def process_image(image):
#     array = np.frombuffer(image.raw_data, dtype=np.uint8)
#     array = array.reshape((image.height, image.width, 4))  # Convert to 2D array
#     image_rgb = array[:, :, :3]  # Discard the alpha channel

#     # Preprocess image for DeepLabv3 (resize and normalize)
#     input_image = cv2.cvtColor(image_rgb, cv2.COLOR_BGR2RGB)
#     input_image = torch.tensor(input_image).float().permute(2, 0, 1).unsqueeze(0) / 255.0

#     # Run the model on the image
#     with torch.no_grad():
#         output = model(input_image)['out'][0]  # Get model's output
#     output_predictions = output.argmax(0)  # Get the class predictions (highest logit)
    
#     # Convert the predictions to colors
#     seg_image = COLORS[output_predictions.numpy()]
    
#     # Create the masked image by blending the segmentation output with the original image
#     masked_image = np.copy(image_rgb)
#     masked_image[output_predictions.numpy() == 0] = [0, 0, 0]  # Optionally mask out background (class 0)

#     # Overlay the segmentation result on the original image
#     blended_image = cv2.addWeighted(image_rgb, 0.7, seg_image.astype(np.uint8), 0.3, 0)
#     # unique_values = np.unique(output_predictions)
#     # Display the result
#     display_queue.put(blended_image)
#     # cv2.imshow("POV + Segmentation", blended_image)

#     # cv2.waitKey(1)


# # normal display without segmentation 
# def process_image(image):
#     # Convert the image to a numpy array
#     image_array = np.array(image.raw_data)
#     image_array = image_array.reshape((image.height, image.width, 4))  # Change to a 3D array (height, width, RGBA)

#     # Convert to BGR (OpenCV format)
#     image_bgr = image_array[:, :, :3]

#     # Use a queue to safely pass images to the display thread
#     display_queue.put(image_bgr)

# def display_image():
#     while True:
#         # Check if the queue has a new image to display
#         if not display_queue.empty():
#             image_bgr = display_queue.get()
#             cv2.imshow('RGB Cam + Segmentation', image_bgr)
#             cv2.waitKey(1)

# # Start the display thread
# import queue
# display_queue = queue.Queue()

# display_thread = threading.Thread(target=display_image)
# display_thread.daemon = True
# display_thread.start()

# camera.listen(lambda image: threading.Thread(target=process_image, args=(image,)).start())


In [11]:
# jake.set_autopilot(True)

In [12]:

# import glob
# import os
# import sys
# import argparse
# import time
# from datetime import datetime
# import random
# import threading
# import numpy as np
# from mayavi import mlab


# def lidar_callback(point_cloud, buf):
#     """Prepares a point cloud with intensity colors and stores in a buffer, which updated the mayavi visualisation"""
#     data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
#     data = np.reshape(data, (int(data.shape[0] / 4), 4))

#     # Isolate the intensity and compute a color for it
#     intensity = data[:, -1]
#     intensity_col = 1.0 - np.log(intensity) / np.log(np.exp(-0.004 * 100))

#     # Isolate the 3D data
#     points = data[:, :-1]

#     # We're negating the y to correclty visualize a world that matches
#     # what we see in Unreal since Mayavi uses a right-handed coordinate system
#     points[:, :1] = -points[:, :1]

#     # # An example of converting points from sensor to vehicle space if we had
#     # # a carla.Transform variable named "tran":
#     # points = np.append(points, np.ones((points.shape[0], 1)), axis=1)
#     # points = np.dot(tran.get_matrix(), points.T).T
#     # points = points[:, :-1]

#     #copy points/intensities into buffer
#     buf['pts'] = points
#     buf['intensity'] = intensity

# def semantic_lidar_callback(point_cloud, buf):
#     """Prepares a point cloud with semantic segmentation colors"""
#     data = np.frombuffer(point_cloud.raw_data, dtype=np.dtype([
#         ('x', np.float32), ('y', np.float32), ('z', np.float32),
#         ('CosAngle', np.float32), ('ObjIdx', np.uint32), ('ObjTag', np.uint32)]))

#     # We're negating the y to correclty visualize a world that matches
#     # what we see in Unreal since Open3D uses a right-handed coordinate system
#     points = np.array([data['x'], -data['y'], data['z']]).T

#     # # An example of adding some noise to our data if needed:
#     # points += np.random.uniform(-0.05, 0.05, size=points.shape)

#     # Colorize the pointcloud based on the CityScapes color palette
#     labels = np.array(data['ObjTag'], dtype=np.float32)
#     int_color = labels

#     # # In case you want to make the color intensity depending
#     # # of the incident ray angle, you can use:
#     #  int_color *= np.array(data['CosAngle'])

#     buf['pts'] = points
#     buf['intensity'] = labels


# def carlaEventLoop(world):
#         frame = 0
#         dt0 = datetime.now()
#         while True:
#             time.sleep(0.005)
#             world.tick()

#             process_time = datetime.now() - dt0
#             sys.stdout.write('\r' + 'FPS: ' + str(1.0 / process_time.total_seconds()))
#             sys.stdout.flush()
#             dt0 = datetime.now()
#             frame += 1
# try:
#     original_settings = world.get_settings()
#     settings = world.get_settings()
#     traffic_manager = client.get_trafficmanager(8000)
#     traffic_manager.set_synchronous_mode(True)

#     delta = 0.05

#     settings.fixed_delta_seconds = delta
#     settings.synchronous_mode = True
#     world.apply_settings(settings)
#     # spawn jake at the left side of the parking lot 

#     jake_location = carla.Location(x=-58.468666, y=188.412048, z=7.642653)
#     jake_rotation = carla.Rotation(pitch=0, yaw=-90, roll=0) 

#     jake_transform = carla.Transform(jake_location, jake_rotation)
#     # Spawn the camera at the spectator's position and orientation
#     jake = world.spawn_actor(vehicle_bp, jake_transform)


#     jake.set_autopilot(True)

#     lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')
#     # lidar_bp.set_attribute('sensor_tick', '5.0')
#     lidar_transform = carla.Transform(carla.Location(x=0, y=5, z=0))  # Adjust if needed

#     lidar = world.spawn_actor(lidar_bp, lidar_transform, attach_to=jake)

#     fig = mlab.figure(size=(960,540), bgcolor=(0.05,0.05,0.05))
#     vis = mlab.points3d(0, 0, 0, 0, mode='point', figure=fig)
#     mlab.view(distance=25)
#     buf = {'pts': np.zeros((1,3)), 'intensity':np.zeros(1)}

#     #  @mlab.animate(delay=100)
#     def anim():
#         i=0
#         while True:
#             vis.mlab_source.reset(x=buf['pts'][:,0], y=buf['pts'][:,1], z=buf['pts'][:,2], scalars=buf['intensity'])
#             mlab.savefig(f'{i}.png', figure=fig)
#             time.sleep(0.1)
#             i+=1

#     # lidar.listen(lambda data: semantic_lidar_callback(data, buf))
#     lidar.listen(lambda data: lidar_callback(data, buf))

#     loopThread = threading.Thread(target=carlaEventLoop, args=[world], daemon=True)
#     loopThread.start()
#     anim()
#     #  mlab.show()

# finally:
#     world.apply_settings(original_settings)
#     traffic_manager.set_synchronous_mode(False)

#     jake.destroy()
#     lidar.destroy()

In [13]:
# from debug_helper import *
# #
# jake_location = carla.Location(x=-58.468666, y=188.412048, z=7.642653)

# draw_line(world, (-40.468666, 193.412048, 1), (-40.468666, 153.412048, 1))
# draw_line(world, (-40.468666, 153.412048, 1), (-70.468666, 153.412048, 1))
# draw_line(world, (-70.468666, 193.412048, 1), (-70.468666, 153.412048, 1))
# draw_line(world, (-40.468666, 193.412048, 1), (-70.468666, 193.412048, 1))


In [14]:
# import time 
# from car_control_helper import * 

# move_car(jake, steer=-1)
# time.sleep(1)
# move_car(jake, steer=0)
# time.sleep(1)
# stop_car(jake)
# # move_car(jake, reverse=True, steer = -1)
# # time.sleep(1)
# # stop_car(jake)

In [15]:
# import numpy as np
# from debug_helper import *
# start_x = -70.5
# end_x = -40.45
# start_y =  153.412048
# end_y = 193.412048
# x_range = np.linspace(start_x, end_x, 100) 
# # y_range = np.linspace(start_y, end_y, steps) # can potentially do two orientations to incraese accuracy
# for x in x_range:
    
#     start = carla.Vector3D(x, start_y, 1)
#     end = carla.Vector3D(x, end_y, 1)
#     draw_line(world, (x, start_y, 1), (x, end_y, 1))
#     res = world.cast_ray(start, end)    
#     for point in res:
#         print(point.location, point.label)


In [16]:
# # test collision detector 
# from hybrid_astar_carla import detect_obstacles

# start_x = -70.5
# end_x = -40.45
# start_y =  153.412048
# end_y = 193.412048
# print(len(detect_obstacles(world, start_x, end_x, start_y, end_y)))

In [17]:
# Attach a LiDAR to jake
import carla
import numpy as np
import time
import random
from debug_helper import *

X_OFFSET = 2.5
Z_OFFSET = 0.7
min_distance_threshold = 2.5
vehicle_transform = jake.get_transform() 

# Function to process and extract all LiDAR points
def lidar_callback(point_cloud):
    print("In")
    # point_cloud is the LiDAR data provided in carla.LidarMeasurement format
    # Extract the x, y, z coordinates from the point cloud (i.e., the 3D points)
    
    points = np.frombuffer(point_cloud.raw_data, dtype=np.float32)
    points = np.reshape(points, (int(len(points) / 4), 4))  # Reshape into (x, y, z, intensity)
    for point in points:
        x, y, z, _ = point
        x = float(x) + X_OFFSET
        y = float(y)
        z = float(z) + Z_OFFSET
        
        distance = np.sqrt(x**2 + y**2 + z**2)  # Compute distance from LiDAR
        angle = np.degrees(np.arctan2(y, x))  # Get angle from front direction
        if not (distance >= min_distance_threshold and -90 <= angle <= 90) or z < 0.1:
            return 
        
        local_point = carla.Location(x=x, y=y, z=z)
        a = vehicle_transform.transform(local_point)
        draw_point(world, [a.x, a.y, a.z], life_time=5)
    return points[:, :3]  # Return only the x, y, z coordinates (ignore intensity)

# Function to spawn a vehicle and LiDAR sensor
def detect_surrounding_with_lidar():

    lidar_bp = blueprint_library.find('sensor.lidar.ray_cast')
    # lidar_bp.set_attribute('channels', '128')  # Use a LiDAR sensor with more channels for a wider FOV
    # lidar_bp.set_attribute('rotation_frequency', '20')  # Increase the rotation frequency
    lidar_bp.set_attribute('range', "30")  
    # lidar_bp.set_attribute('dropoff_general_rate', "0")  
    # lidar_bp.set_attribute('dropoff_zero_intensity', "0")  

    lidar_transform = carla.Transform(carla.Location(x=X_OFFSET, y=0, z=Z_OFFSET)) 
    lidar = world.spawn_actor(lidar_bp, lidar_transform, attach_to=jake)
    # jake.set_autopilot(True)
    lidar.listen(lambda data: lidar_callback(data)) 
    time.sleep(0.1)  # Collect data for 5 seconds

    lidar.stop()
    lidar.destroy()
"""
Notes:
1. The lidar also detects the ground as obstacles, so we need to ignore the points
that have z = 0 or something
2. In 0.9.15 
"""
all_actors = world.get_actors()

# for actor in all_actors:
#     print( actor.type_id)
#     if "walker.pedestrian." in actor.type_id:
#         print("Found pedestrian:", actor)
#         actor.set_collisions(True)
#         actor.set_simulate_physics(True)
detect_surrounding_with_lidar()

In
In
In
In
In
In
In
In
In
In


In [18]:
from debug_helper import *
draw_line(world, [-48.5,  180, 0.05], [-48.5, 183, 0.05], carla.Color(255, 216, 0), life_time=30)
draw_line(world, [-48.5,  180, 0.05], [-53.5, 180, 0.05], carla.Color(255, 216, 0), life_time=30)
draw_line(world, [-48.5,  183, 0.05], [-53.5, 183, 0.05], carla.Color(255, 216, 0), life_time=30)


In [19]:
# call hybrid A*
import numpy as np
from hybrid_astar_carla import hybrid_astar_planning, detect_obstacles
import time
from debug_helper import *

# bounding box

# The way the algorithm is implemented requires all points to be positive, so we need to offset
offset_x = 73 
offset_y = -160

start_x = -70.5 
end_x = -40.45 
start_y =  153.412048
end_y = 193.412048
destroy_all_prius(world)
time.sleep(1)
oox, ooy = detect_obstacles(world, start_x, end_x, start_y, end_y)
time.sleep(1)
jake = spawn_jake()

for i in range(len(oox)):
    # draw_point(world, [ox[i], oy[i], 1])
    print( [oox[i], ooy[i] , 1])
    draw_point(world, [oox[i], ooy[i] , 1], life_time=30)
    # oox[i] += offset_x
    # ooy[i] += offset_y



[-68.07171630859375, 176.2527313232422, 1]
[-65.94696807861328, 169.64183044433594, 1]
[-68.07171630859375, 185.01856994628906, 1]
[-48.78352355957031, 176.4423370361328, 1]
[-48.78352355957031, 176.8463897705078, 1]
[-70.45552825927734, 183.31103515625, 1]
[-70.45552825927734, 183.71507263183594, 1]
[-67.46464538574219, 176.248291015625, 1]
[-69.61701965332031, 170.38172912597656, 1]
[-67.46464538574219, 185.01856994628906, 1]
[-49.25252151489258, 165.450439453125, 1]
[-52.287879943847656, 159.9148406982422, 1]
[-69.89292907714844, 176.05714416503906, 1]
[-70.5, 166.34133911132812, 1]
[-70.5, 166.745361328125, 1]
[-69.89292907714844, 185.01856994628906, 1]
[-65.6434326171875, 182.5814208984375, 1]
[-70.5, 175.230224609375, 1]
[-70.5, 175.63427734375, 1]
[-69.99654388427734, 184.9272003173828, 1]
[-70.19646453857422, 179.6923828125, 1]
[-48.96818161010742, 159.06861877441406, 1]
[-69.96039581298828, 175.63427734375, 1]
[-68.07171630859375, 171.15194702148438, 1]
[-49.55605697631836, 16

In [20]:

def spawn_pedestrian(world, start_x, start_y):
    # Get the walker blueprints
    # 0048 for baby
    walker_bp = random.choice(world.get_blueprint_library().filter('walker.pedestrian.*'))
    
    # Define spawn points for the pedestrian

    ped_loc = carla.Location(x=start_x, y=start_y, z=1)

    ped_transform = carla.Transform(ped_loc, carla.Rotation())
    # Spawn the pedestrian
    walker = world.spawn_actor(walker_bp, ped_transform)


    return walker

def destroy_all_peds(world):
    
    actors = world.get_actors()
    peds = [actor for actor in actors if actor.type_id.startswith("walker.pedestrian")]

    for ped in peds:
        ped.destroy()




In [25]:
destroy_all_peds(world)


In [26]:
# # spawn a pedestrian in the way and see how the car react
ped_x, ped_y = -53.468666, 173.412048
oox.append(ped_x + offset_x)
ooy.append( ped_y + offset_y)

In [56]:
import threading

def calculate_direction(current_location, target_location):
    direction = carla.Vector3D(
        x=target_location.x - current_location.x,
        y=target_location.y - current_location.y,
        z=0  # Ignore vertical movement
    )
    # Normalize the direction vector
    length = math.sqrt(direction.x**2 + direction.y**2)
    if length > 0:
        direction.x /= length
        direction.y /= length
    return direction

# Function to move the pedestrian between two points
def patrol(pedestrian, point1, point2, speed=2.4, num_patrol=100):
    # num 
    control = carla.WalkerControl()
    control.speed = speed
    control.jump = False

    for i in range(num_patrol):
        # Move to point1
        time.sleep(1)
        current_location = pedestrian.get_location()

        direction = calculate_direction(current_location, point2)
        control.direction = direction
        control.speed = speed
        pedestrian.apply_control(control)

        # One bug here is if there's obstacles in the 
        # middle the pedestrian will get stuck and walk forever
        while current_location.distance(point2) > 0.5:
            time.sleep(0.1)
            current_location = pedestrian.get_location()

        # Stop the pedestrian briefly
        control.speed = 0
        pedestrian.apply_control(control)
        time.sleep(1)

        print(current_location.distance(point1))
        # Move to point2
        current_location = pedestrian.get_location()
        direction = calculate_direction(current_location, point1)
        control.direction = direction
        control.speed = speed
        pedestrian.apply_control(control)

        while current_location.distance(point1) > 0.5:
            time.sleep(0.1)
            current_location = pedestrian.get_location()

        control.speed = 0
        pedestrian.apply_control(control)
        time.sleep(1)

place_to_coords = {
    "cornerTopLeft" : (-65.468666, 188.412048),
    "cornerTopRight" : (-65.468666, 157.412048),
    "middle1" :  (-55.468666, 168.412048),
    "middle2" :  (-57.468666, 168.412048),
    "middle3" :  (-59.468666, 168.412048),
}

direction = {
    "diagonalTopRight" : (1, 1),
    "diagonalTopLeft" : (-1, 1),
    "diagonalBottomRight" : (1, -1),
    "diagonalBottomLeft" : (-1, -1),
}

destroy_all_peds(world)
cornerTopLeft = place_to_coords["cornerTopLeft"]
# Spawn the pedestrian at the top left and make it move to the bottom right
pedestrian = spawn_pedestrian(world, cornerTopLeft[0], cornerTopLeft[1])
point1 = carla.Location(x=cornerTopLeft[0], y=cornerTopLeft[1], z=1)
point2 = carla.Location(x=cornerTopLeft[0] + 20, y=cornerTopLeft[1] - 20, z=1)
patrol_thread = threading.Thread(target=patrol, args=(pedestrian, point1, point2, 2.3))
patrol_thread.daemon = True 
patrol_thread.start()

cornerTopRight = place_to_coords["cornerTopRight"]
# Spawn the pedestrian at the top Right and make it move to the bottom left
pedestrian = spawn_pedestrian(world, cornerTopRight[0], cornerTopRight[1])
point1 = carla.Location(x=cornerTopRight[0], y=cornerTopRight[1], z=1)
point2 = carla.Location(x=cornerTopRight[0] + 15, y=cornerTopRight[1] + 15, z=1)
patrol_thread = threading.Thread(target=patrol, args=(pedestrian, point1, point2, 1.5))
patrol_thread.daemon = True 
patrol_thread.start()

middle = place_to_coords["middle1"]
# Spawn the pedestrian at the center and make it move left and right
pedestrian = spawn_pedestrian(world, middle[0], middle[1])
point1 = carla.Location(x=middle[0], y=middle[1], z=1)
point2 = carla.Location(x=middle[0], y=middle[1] - 10, z=1)
patrol_thread = threading.Thread(target=patrol, args=(pedestrian, point1, point2, 1.5))
patrol_thread.daemon = True 
patrol_thread.start()


middle = place_to_coords["middle2"]
# Spawn the pedestrian at the center and make it move left and right
pedestrian = spawn_pedestrian(world, middle[0], middle[1])
point1 = carla.Location(x=middle[0], y=middle[1], z=1)
point2 = carla.Location(x=middle[0], y=middle[1] + 10, z=1)
patrol_thread = threading.Thread(target=patrol, args=(pedestrian, point1, point2, 1.5))
patrol_thread.daemon = True 
patrol_thread.start()


# pedestrian.apply_control(control)
# time.sleep(2)

# control.speed = 0
# pedestrian.apply_control(control)
# time.sleep(1)

9.614583969116211
9.607940673828125
28.1827335357666
20.7667293548584
9.58944034576416
9.615158081054688
9.58109188079834
9.589447975158691
28.2137393951416
20.764572143554688
9.554953575134277
9.583755493164062
28.088058471679688
9.530509948730469
9.584182739257812
20.74277114868164
9.512947082519531
9.56292724609375
28.242549896240234
9.515938758850098
9.553040504455566
20.756675720214844
9.604866027832031
9.564376831054688
28.085317611694336
9.56427001953125
9.617805480957031
20.842342376708984
9.57745361328125
9.639945030212402
28.189151763916016


In [23]:
from debug_helper import *
destroy_all_peds(world)

# Make the pedestrain walk
patrol_points = [
    carla.Location(x=-58.468666, y=183.412048, z=1),
    carla.Location(x=-57.468666, y=183.412048, z=1),
]
settings = world.get_settings()
settings.fixed_delta_seconds = 0.05
settings.synchronous_mode = True
world.apply_settings(settings)
world.tick()
pedestrian = spawn_pedestrian(world, ped_x, ped_y)
controller_bp = world.get_blueprint_library().find('controller.ai.walker')
controller = world.spawn_actor(controller_bp, carla.Transform(), pedestrian)
random_loc = world.get_random_location_from_navigation()
controller.set_max_speed(2.0) 

def distance(location1, location2):
    return math.sqrt((location1.x - location2.x)**2 + (location1.y - location2.y)**2)

def walk_to(point):
    draw_point(world, [point.x, point.y, point.z])
    # world.tick()
    controller.start()
    # world.tick()
    print("Hello world")
    controller.go_to_location(point)
    while distance(pedestrian.get_location(), point) > 0.5:
        print("Distance:", distance(pedestrian.get_location(), point))
        world.tick()
        time.sleep(0.05)
try:
    walk_to(patrol_points[1])
    settings.fixed_delta_seconds = None
    settings.synchronous_mode = False
    world.apply_settings(settings)
    world.wait_for_tick()
except Exception as e:
    print(f"Error in go_to_location: {e}")

Hello world
Distance: 192.20464889499382
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distance: 10.04987562112089
Distanc

KeyboardInterrupt: 

: 

In [ ]:
import copy

# for i in range(len(oox)):
#     # draw_point(world, [ox[i], oy[i], 1])
#     draw_point(world, [oox[i] - offset_x, ooy[i] - offset_y, 1], life_time=30)
ox = copy.deepcopy(oox)
oy = copy.deepcopy(ooy)

# First kill jake before we detect obstacles

# start_x += offset_x
# end_x += offset_x
# start_y += offset_y
# end_y += offset_y

# -90 is right, 0 is down, 90 is left, 180 is top

# 0 degree is south
# -90 degree is east
# 180 degree is north
sx, sy, syaw0 = -58.468666 + offset_x, 188.412048 + offset_y, np.deg2rad(-90.0) 
# goal_location = carla.Location(x=-50.468666, y=174.412048, z=7.642653)
# goal_rotation = carla.Rotation(pitch=0, yaw=0, roll=0) 
# gx, gy, gyaw0 = -51.468666 + offset_x, 171.4 + offset_y, np.deg2rad(0.0)
gx, gy, gyaw0 = -51.468666 + offset_x, 180.4 + offset_y, np.deg2rad(0.0)
# gx, gy, gyaw0 = -50.468666 + offset_x, 190.4 + offset_y, np.deg2rad(180.0)
# gx, gy, gyaw0 = -55.468666 + offset_x, 179.4 + offset_y, np.deg2rad(0.0)
# gx, gy, gyaw0 = -45.468666 + offset_x, 173.4 + offset_y, np.deg2rad(0.0)
# gx, gy, gyaw0 = -67.468666 + offset_x, 177.7 + offset_y, np.deg2rad(0.0)
# gx, gy, gyaw0 = -67.068666 + offset_x, 167 + offset_y, np.deg2rad(180.0)
draw_point(world, [sx - offset_x, sy - offset_y, 1])
draw_point(world, [gx - offset_x, gy - offset_y, 1], carla.Color(0, 255, 0), life_time=30)

# Decrease this to increase accuracy
XY_RESO = 1.0  # [m]
# Decrease this to increase accuracy
YAW_RESO = np.deg2rad(5.0)  # [rad]

t0 = time.time()
path, closed_set = hybrid_astar_planning(world, sx, sy, syaw0, gx, gy, gyaw0,
                        ox, oy, XY_RESO, YAW_RESO)
t1 = time.time()
print(closed_set)
print("running T: ", t1 - t0)
if path is not None:
    for i in range(len(path.x)):
        draw_point(world, [path.x[i] - offset_x, path.y[i] - offset_y, 1], life_time=30)
        print(path.x[i] - offset_x, path.y[i] - offset_y, path.yaw[i], path.direction[i])
else:
    print("no valid path found")



In [ ]:
import copy

# for i in range(len(oox)):
#     # draw_point(world, [ox[i], oy[i], 1])
#     draw_point(world, [oox[i], ooy[i], 1], life_time=30)
ox = copy.deepcopy(oox)
oy = copy.deepcopy(ooy)

# First kill jake before we detect obstacles

# start_x 
# end_x 
# start_y 
# end_y 

# -90 is right, 0 is down, 90 is left, 180 is top

# 0 degree is south
# -90 degree is east
# 180 degree is north
sx, sy, syaw0 = -58.468666, 188.412048, np.deg2rad(-90.0) 
# goal_location = carla.Location(x=-50.468666, y=174.412048, z=7.642653)
# goal_rotation = carla.Rotation(pitch=0, yaw=0, roll=0) 
# gx, gy, gyaw0 = -51.468666, 171.4, np.deg2rad(0.0)
gx, gy, gyaw0 = -51.468666, 180.4, np.deg2rad(0.0)
# gx, gy, gyaw0 = -50.468666, 190.4, np.deg2rad(180.0)
# gx, gy, gyaw0 = -55.468666, 179.4, np.deg2rad(0.0)
# gx, gy, gyaw0 = -45.468666, 173.4, np.deg2rad(0.0)
# gx, gy, gyaw0 = -67.468666, 177.7, np.deg2rad(0.0)
# gx, gy, gyaw0 = -67.068666, 167, np.deg2rad(180.0)
draw_point(world, [sx, sy, 1])
draw_point(world, [gx, gy, 1], carla.Color(0, 255, 0), life_time=30)

# Decrease this to increase accuracy
XY_RESO = 1.0  # [m]
# Decrease this to increase accuracy
YAW_RESO = np.deg2rad(5.0)  # [rad]

t0 = time.time()
path, closed_set = hybrid_astar_planning(world, sx, sy, syaw0, gx, gy, gyaw0,
                        ox, oy, XY_RESO, YAW_RESO)
t1 = time.time()
print(closed_set)
print("running T: ", t1 - t0)
if path is not None:
    for i in range(len(path.x)):
        draw_point(world, [path.x[i], path.y[i], 1], life_time=30)
        # print(path.x[i], path.y[i], path.yaw[i], path.direction[i])
else:
    print("no valid path found")



In [ ]:
import copy
from hybrid_astar_carla import hybrid_astar_planning_online

start_x = -70.5 
end_x = -40.45 
start_y =  153.412048
end_y = 193.4120
sx, sy, syaw0 = -58.468666, 188.412048, np.deg2rad(-90.0) 
# goal_location = carla.Location(x=-50.468666, y=174.412048, z=7.642653)
# goal_rotation = carla.Rotation(pitch=0, yaw=0, roll=0) 
# gx, gy, gyaw0 = -51.468666, 171.4, np.deg2rad(0.0)
gx, gy, gyaw0 = -51.468666, 180.4, np.deg2rad(0.0)
# gx, gy, gyaw0 = -50.468666, 190.4, np.deg2rad(180.0)
# gx, gy, gyaw0 = -55.468666, 179.4, np.deg2rad(0.0)
# gx, gy, gyaw0 = -45.468666, 173.4, np.deg2rad(0.0)
# gx, gy, gyaw0 = -67.468666, 177.7, np.deg2rad(0.0)
# gx, gy, gyaw0 = -67.068666, 167, np.deg2rad(180.0)
draw_point(world, [sx, sy, 1])
draw_point(world, [gx, gy, 1], carla.Color(0, 255, 0), life_time=30)

# Decrease this to increase accuracy
XY_RESO = 1.0  # [m]
# Decrease this to increase accuracy
YAW_RESO = np.deg2rad(5.0)  # [rad]

t0 = time.time()
path, closed_set = hybrid_astar_planning_online(world, jake, sx, sy, syaw0, gx, gy, gyaw0, XY_RESO, YAW_RESO)
t1 = time.time()
if path is not None:
    for i in range(len(path.x)):
        draw_point(world, [path.x[i], path.y[i], 1], life_time=30)
        # print(path.x[i], path.y[i], path.yaw[i], path.direction[i])
else:
    print("no valid path found")



In [ ]:
points = [[]]

In [ ]:
# from car_control_helper import * 

# Calculate car's parameters
# sx, sy = -58.468666, 188.412048

# physics_control = jake.get_physics_control()
# vehicle_transform = jake.get_transform()
# vehicle_bbox = jake.bounding_box
# # Get the position (center of the bounding box)
# vehicle_location = vehicle_transform.location
# wheels = physics_control.wheels
# wheel_FL = wheels[0]
# wheel_FR = wheels[1]
# wheel_RL = wheels[2]
# wheel_RR = wheels[3]
# print(wheel_FL.max_steer_angle)
# print(wheel_FR.max_steer_angle)
# print(wheel_RL.max_steer_angle)
# print(wheel_RR.max_steer_angle)
# length = vehicle_bbox.extent.x * 2
# width = vehicle_bbox.extent.y * 2
# height = vehicle_bbox.extent.z * 2
# # print(length, width, height)
# draw_line(world, ( wheel_FL.position.x / 100 - 0.1,  wheel_FL.position.y / 100, 0), ( wheel_FL.position.x / 100 + 0.1,  wheel_FL.position.y / 100, 0))

# # draw_line(world, (sx, sy + length / 2, 1), (sx, sy - length / 2, 1))
# # draw_line(world, (sx + width / 2, sy, 1), (sx - width / 2, sy, 1))
# # print(wheel_FL.position, wheel_FL.radius)
# # draw_point(world, [sx,  wheels_RL.position.y / 100, 3])
# # draw_point(world, [sx, sy + length / 2 , 3])
# # print( sy - length / 2 -  wheels_RL.position.y / 100)
# # print(  sy + length / 2 -  wheels_RL.position.y / 100   )
# # print( wheel_FL.position.x / 100 -  wheel_FR.position.x / 100) # DISTANCE BETWEEN LEFT AND RIGHT WHELS
# # print( wheel_RL.position.x / 100 -  wheel_RR.position.x / 100)
# # print( wheel_RL.position.y / 100 -  wheel_FL.position.y / 100) # WHEEL BASE
# # print( wheel_RR.position.y / 100 -  wheel_FR.position.y / 100)


In [ ]:

# from car_control_helper import *
# import time 
# import copy
# import math

# class WaypointFollower:
#     def __init__(self, vehicle, waypoints):
#         self.vehicle = vehicle
#         self.waypoints = waypoints
#         self.target_idx = 0

#     def get_vehicle_state(self):
#         transform = self.vehicle.get_transform()
#         velocity = self.vehicle.get_velocity()
#         speed = math.sqrt(velocity.x**2 + velocity.y**2)
#         return transform, speed
    
#     def compute_steering(self, target_wp, vehicle_transform):
#         vehicle_location = vehicle_transform.location
#         vehicle_yaw = math.radians(vehicle_transform.rotation.yaw)
        
#         # if  target_wp[3] == -1:
#         #     vehicle_yaw += math.pi
#         # Compute relative position
#         dx = target_wp[0] - vehicle_location.x
#         dy = target_wp[1] - vehicle_location.y
#         target_yaw = math.atan2(dy, dx)
#         # Calculate angle error and normalize
#         angle_error = target_yaw - vehicle_yaw
#         # angle_error = (angle_error + math.pi) % (2 * math.pi) - math.pi
        
#         if target_wp[3] == -1:
#             angle_error += math.pi  # Face opposite direction when reversing
        
#         # Apply proportional gain (adjust Kp as needed)
#         Kp = 1  # Tune this value for better steering response
#         steer = Kp * angle_error
#         # Compute steering error
#         return steer

#     def compute_steering2(self, target_wp, vehicle_transform):
#         # target_wp is assumed to be a tuple like (x, y, yaw, direction)
#         target_x, target_y, target_wp_yaw, _ = target_wp  # Extract waypoint yaw
        
#         # Convert the vehicle's current yaw to radians
#         vehicle_yaw = math.radians(vehicle_transform.rotation.yaw)
        
#         # Adjust the desired yaw when reverd
#         if target_wp[3] == -1:
#             desired_yaw = (target_wp_yaw + math.pi) % (2 * math.pi)
#         else:
#             desired_yaw = target_wp_yaw
        
#         # Compute the error between the vehicle's current yaw and the desired yaw
#         angle_error = desired_yaw - vehicle_yaw
#         # Normalize the error to [-pi, pi]
#         angle_error = (angle_error + math.pi) % (2 * math.pi) - math.pi
        
#         return 3 * angle_error

#     def control_vehicle(self):
#         while True:
#             if self.target_idx >= len(self.waypoints):
#                 print("All waypoints reached!")
#                 control = carla.VehicleControl(throttle=throttle, steer=steer, brake=brake)
#                 self.vehicle.apply_control(control)
#                 return 

#             transform, speed = self.get_vehicle_state()
#             target_wp = self.waypoints[self.target_idx]
#             target_x, target_y, target_yaw, direction = target_wp
    
#             car_x, car_y = transform.location.x, transform.location.y
#             draw_point(world, [car_x, car_y , 1], carla.Color(255, 255, 0), life_time=30)
            
#             # Compute steering
#             steer = self.compute_steering2(target_wp, transform)
#             # if direction == -1:
#             #     steer *= -1
                
#             # vehicle_location = transform.location
#             # vehicle_yaw = math.radians(transform.rotation.yaw)
#             # print(target_yaw, vehicle_yaw)
#             # steer = (target_yaw - vehicle_yaw)
#             # print("Steer is", steer)
#             steer = max(-1.0, min(1.0, steer))  # Limit steering range
#             # Determine speed and control
#             throttle = 0.2  
#             brake = 0.0 if abs(throttle) > 0.1 else 1.0

#             # Update target if close enough
#             vehicle_location = transform.location
#             dist_to_target = math.sqrt((target_x - vehicle_location.x) ** 2 + (target_y - vehicle_location.y) ** 2)

#             if dist_to_target < 0.2:  # Move to next waypoint
#                 self.target_idx += 1
#                 stop_car(self.vehicle)
#                 print(f"Target idx {self.target_idx}")

#             control = carla.VehicleControl(throttle=throttle, reverse=direction == -1, steer=steer, brake=brake)
#             self.vehicle.apply_control(control)
#             time.sleep(0.05)

# stop_car(jake)
# time.sleep(1)
# transform = carla.Transform(
#     carla.Location(x=path.x[0] - offset_x, y=path.y[0] - offset_y, z=0.3),
#     carla.Rotation(yaw=math.degrees(path.yaw[0]))
# )
# jake.set_transform(transform)
# time.sleep(2)

# path2 = copy.deepcopy(path)

# for i in range(len(path2.x)):
#     path2.x[i] -= offset_x
#     path2.y[i] -= offset_y

# try:

#     waypoints = [(10, 20, 90, 1), (15, 25, 180, -1), (20, 30, 270, 1)]  # (x, y, yaw, direction)
#     waypoints = list(zip(path2.x, path2.y, path2.yaw, path2.direction))
#     follower = WaypointFollower(jake, waypoints)
#     control = follower.control_vehicle()
# finally:
#     stop_car(jake)

In [ ]:
# import carla
# import random
# import time

# # Choose a random walker blueprint
# walker_bp = random.choice(blueprint_library.filter("walker.pedestrian.*"))

# # Define spawn points
# spawn_points = world.get_map().get_spawn_points()
# point_a = carla.Location(x=-58.468666, y=178.412048, z=0.1)
# point_b = carla.Location(x=-53.468666, y=188.412048, z=0.1)
# draw_point(world, [-58.468666, 178.412048, 1], carla.Color(255, 0, 0), life_time=30)
# draw_point(world, [-53.468666, 188.412048, 1], carla.Color(255, 0, 0), life_time=30)
# # Get a pedestrian blueprint
# walker_bp = world.get_blueprint_library().filter('walker.pedestrian.*')[0]

# # Spawn the pedestrian at point A
# walker = world.spawn_actor(walker_bp, carla.Transform(point_a, carla.Rotation()))
# # Get the AI controller blueprint
# controller_bp = world.get_blueprint_library().find('controller.ai.walker')

# # Spawn and attach the controller
# controller = world.spawn_actor(controller_bp, carla.Transform(), walker)
# controller.start()  # Start the controller

# print("get_random_location_from_navigation", world.get_random_location_from_navigation())
# print("type", type(world.get_random_location_from_navigation()))
# controller.go_to_location(world.get_random_location_from_navigation())

In [ ]:
# current_target = point_b
# controller.go_to_location(current_target)
# threshold = 1.0  # Distance threshold to switch target (meters)

# try:
#     while True:
#         world.wait_for_tick()  # Sync with CARLA's simulation time
#         print("asdasdas")
#         if not walker.is_alive or not controller.is_alive:
#             print("Walker or controller has been destroyed.")
#             break
#         current_location = walker.get_location()
#         distance = current_location.distance(current_target)

#         if distance < threshold:
#             # Switch target
#             current_target = point_b if current_target == point_a else point_a
#             controller.go_to_location(current_target)
        
#         time.sleep(0.1)  # Check every 0.1 seconds

# except KeyboardInterrupt:
#     # Cleanup
#     controller.stop()
#     walker.destroy()
#     controller.destroy()

In [ ]:
# try to drive there, don't crash into baby

from car_control_helper import *

# desired_yaw 1.487449083002621, current_yaw -1.6678677495799579, yaw_error -3.127868474597008
# desired_yaw 1.4445045169153057, current_yaw -1.6678724101118654, yaw_error 3.112376927027171
# desired_yaw 1.4731006093480203, current_yaw -1.6678718774796475, yaw_error 3.1409724868276676
# desired_yaw 1.470253600251676, current_yaw -1.6678686816863393, yaw_error 3.1381222819380152

try:
    stop_car(jake)
    follow_path_old(world, jake, path, 0, 0)
finally:
    stop_car(jake)

In [ ]:
# from pure_pursuit import * 
# from car_control_helper import *
# import copy 

# # Pure pursuit algorithm 
# THRESHOLD = 0.5
# def main():

#     cur_path = copy.deepcopy(path)
#     for i in range(len(cur_path.x)):
#         cur_path.x[i] -= offset_x
#         cur_path.y[i] -= offset_y
#         cur_path.yaw[i] -= 90

#     # Teleport to initial position
#     stop_car(jake)
#     time.sleep(1)
#     transform = carla.Transform(
#         carla.Location(x=cur_path.x[0], y=cur_path.y[0], z=0.3),
#         carla.Rotation(yaw=cur_path.yaw[0])
#     )
#     jake.set_transform(transform)

#     x, y, yaw, direct = cur_path.x, cur_path.y, cur_path.yaw, cur_path.direction

#     maxTime = 100.0
#     t = 0.0
#     dt = C.dt  

#     # Initialize the vehicle's state
#     x0 = cur_path.x[0]
#     y0 = cur_path.y[0]
#     yaw0 = cur_path.yaw[0] 
#     direct0 = cur_path.direction[0]
#     node = Node(x=x0, y=y0, yaw=yaw0, v=0.0, direct=direct0)

#     # Build a reference trajectory from the path coordinates.
#     ref_trajectory = PATH(x, y)
#     target_ind, _ = ref_trajectory.target_index(node)
#     print(x)
#     print(y)
#     print(x0)
#     print(y0)
#     try:
#         while t <= maxTime:
#             # Check if we've reached the destination (using a distance threshold).
#             dist_to_goal = math.hypot(node.x - x[-1], node.y - y[-1])
#             if dist_to_goal < C.dist_stop:
#                 print(dist_to_goal, C.dist_stop)
#                 print("Destination reached.")
#                 stop_car(jake)
#                 break

#             # Set target speed based on the direction (or other criteria).
#             if direct0 > 0:
#                 target_speed = 5.0 / 3.6  # Convert km/h to m/s
#                 C.Ld = 4.0
#                 C.dist_stop = 0.3
#                 C.dc = -1.1
#             else:
#                 target_speed = 3.0 / 3.6
#                 C.Ld = 2.5
#                 C.dist_stop = 0.1
#                 C.dc = 0.2

#             # Compute acceleration (from PID control) and steering angle (from pure pursuit)
#             acceleration = pid_control(target_speed, node.v, dist_to_goal, direct0)
#             delta, target_ind = pure_pursuit(node, ref_trajectory, target_ind)

#             # Convert the acceleration command to throttle and brake values.
#             # This is a simple mapping—you might need to refine it for your vehicle.
#             if acceleration >= 0:
#                 throttle = min(acceleration / 2.0, 1.0)  # scale positive acceleration
#                 brake = 0
#             else:
#                 throttle = 0
#                 brake = min(-acceleration / 2.0, 1.0)  # scale deceleration to a brake value

#             # Apply the control commands.
#             # If braking, you might want to use a dedicated function or modify move_car.
#             if brake > 0:
#                 stop_car(jake)  # For simplicity, using stop_car to apply full braking.
#             else:
#                 move_car(jake, reverse=False, steer=-delta, speed=throttle)
#                 time.sleep(0.1)

#             # (Optional) Update the node's state based on the computed commands.
#             # This is your simulation model for state evolution.
#             node.update(acceleration, delta, direct0)

#             # Increment time.
#             t += dt

#     except KeyboardInterrupt:
#         stop_car(jake)
#     finally:
#         stop_car(jake)

# main()